# Calculate the magnitude limit of a K2 tpf

This function seeds a tpf with a grid of pixels with fake events. A numpy array is saved containing the magnitude limit for each pixel in the tpf.

In [ ]:
def K2MagLimit(file):

    path = '/Users/ryanr/Documents/PhD/coding/Kepler/pipelines/Injections/MagLim/'


    Mag = np.arange(19,24,0.5) # distance in Mpc
    
    pixfile = file
    hdu = fits.open(pixfile)
    dat = hdu[1].data
    data = fits.ImageHDU(hdu[1].data.field('FLUX')[:]).data#np.copy(testdata)#
    time = dat["TIME"] + 2454833.0
    Qual = hdu[1].data.field('QUALITY')

    funny_keywords = {'1CTYP4': 'CTYPE1',
                      '2CTYP4': 'CTYPE2',
                      '1CRPX4': 'CRPIX1',
                      '2CRPX4': 'CRPIX2',
                      '1CRVL4': 'CRVAL1',
                      '2CRVL4': 'CRVAL2',
                      '1CUNI4': 'CUNIT1',
                      '2CUNI4': 'CUNIT2',
                      '1CDLT4': 'CDELT1',
                      '2CDLT4': 'CDELT2',
                      '11PC4': 'PC1_1',
                      '12PC4': 'PC1_2',
                      '21PC4': 'PC2_1',
                      '22PC4': 'PC2_2'}
    mywcs = {}
    for oldkey, newkey in funny_keywords.items():
        mywcs[newkey] = hdu[1].header[oldkey]
    wcs = WCS(mywcs)


    Detections = np.zeros((len(Mag),data.shape[1],data.shape[2]))
    Seeded = np.zeros((len(Mag),data.shape[1],data.shape[2]))
    Limit = np.zeros((data.shape[1],data.shape[2]))
    # arrays to initialise multiple concurrent tests
    x = [np.arange(0,data.shape[1],2),np.arange(1,data.shape[1],2)]
    y = [np.arange(0,data.shape[2],2),np.arange(1,data.shape[2],2)]
    #t = randint(100,data.shape[0]-100,size=(data.shape[2]))
    Start = randint(0,data.shape[0]-1)
    for k in range(len(Mag)):
        
        Seed = K2Kilonova(-1,Mag[k],time,Start)

        for i in range(2):
            for j in range(2):
                X, Y = np.meshgrid(x[i],y[j],indexing='ij')
                testdata = np.copy(data)
                testdata[Start:,X,Y] += Seed[Start:,None,None]
                detec = K2tranPix(testdata,time,Qual,wcs,X,Y,Start)
                testdata=[]
                Seeded[k,X,Y] += 1
                if (detec == 1).any:
                    Detections[k,np.where(detec == 1)[0],np.where(detec == 1)[1]] += 1
        #plt.figure()
        #plt.imshow(Detections[k],origin = 'lower')       
    for i in range(data.shape[1]):
        for j in range(data.shape[2]):
            Index = np.where(Detections[:,i,j] == 1)[0]
            if len(Index) > 0:
                Limit[i,j] = Mag[Index[-1]]
    Limit[Limit == 0] = np.nan
    save = path+file.split('K2/')[-1].split('-')[0]
    
    try:
        
        if not os.path.exists(path+file.split('K2/')[-1]):
            os.makedirs(path+file.split('K2/')[-1].split('ktwo')[0])
        np.savez(save,Limit)
    except FileExistsError:
        np.savez(save,Limit)
        
    np.save(save,Limit)
    return

## Testing the dependence on start time

In [ ]:
def K2MagLimitTime(file,Start):

    path = '/Users/ryanr/Documents/PhD/coding/Kepler/pipelines/Injections/MagLim/TimeTest/KSN2015K/'


    Mag = np.arange(19,24,0.5) # distance in Mpc
    
    pixfile = file
    hdu = fits.open(pixfile)
    dat = hdu[1].data
    data = fits.ImageHDU(hdu[1].data.field('FLUX')[:]).data#np.copy(testdata)#
    time = dat["TIME"] + 2454833.0
    Qual = hdu[1].data.field('QUALITY')

    funny_keywords = {'1CTYP4': 'CTYPE1',
                      '2CTYP4': 'CTYPE2',
                      '1CRPX4': 'CRPIX1',
                      '2CRPX4': 'CRPIX2',
                      '1CRVL4': 'CRVAL1',
                      '2CRVL4': 'CRVAL2',
                      '1CUNI4': 'CUNIT1',
                      '2CUNI4': 'CUNIT2',
                      '1CDLT4': 'CDELT1',
                      '2CDLT4': 'CDELT2',
                      '11PC4': 'PC1_1',
                      '12PC4': 'PC1_2',
                      '21PC4': 'PC2_1',
                      '22PC4': 'PC2_2'}
    mywcs = {}
    for oldkey, newkey in funny_keywords.items():
        mywcs[newkey] = hdu[1].header[oldkey]
    wcs = WCS(mywcs)


    Detections = np.zeros((len(Mag),data.shape[1],data.shape[2]))
    Seeded = np.zeros((len(Mag),data.shape[1],data.shape[2]))
    Limit = np.zeros((data.shape[1],data.shape[2]))
    # arrays to initialise multiple concurrent tests
    x = [np.arange(0,data.shape[1],2),np.arange(1,data.shape[1],2)]
    y = [np.arange(0,data.shape[2],2),np.arange(1,data.shape[2],2)]
    #t = randint(100,data.shape[0]-100,size=(data.shape[2]))
    for k in range(len(Mag)):
        
        Seed = KSN2015K(Mag[k],Start,time)#K2Kilonova(-1,Mag[k],time,Start)

        for i in range(2):
            for j in range(2):
                X, Y = np.meshgrid(x[i],y[j],indexing='ij')
                testdata = np.copy(data)
                testdata[Start:,X,Y] += Seed[Start:,None,None]
                detec = K2tranPix(testdata,time,Qual,wcs,X,Y,Start)
                testdata=[]
                Seeded[k,X,Y] += 1
                if (detec == 1).any:
                    Detections[k,np.where(detec == 1)[0],np.where(detec == 1)[1]] += 1
        #plt.figure()
        #plt.imshow(Detections[k],origin = 'lower')       
    for i in range(data.shape[1]):
        for j in range(data.shape[2]):
            Index = np.where(Detections[:,i,j] == 1)[0]
            if len(Index) > 0:
                Limit[i,j] = Mag[Index[-1]]
    Limit[Limit == 0] = np.nan
    save = path+file.split('/')[-1].split('-')[0]+str(Start)
    
    try:
        
        if not os.path.exists(path+file.split('/')[-1]):
            os.makedirs(path+file.split('/')[-1].split('ktwo')[0])
        np.save(save,Limit)
    except FileExistsError:
        np.save(save,Limit)
        
    
    return

## Test the time limit

In [1]:
%run '/Users/ryanr/Documents/PhD/coding/Kepler/pipelines/Injections/Step function test.ipynb'
def K2MagLimitDuration(file,Start):

    path = '/Users/ryanr/Documents/PhD/coding/Kepler/pipelines/Injections/MagLim/Duration/'


    Mag = np.arange(19,25,1) # distance in Mpc
    Duration = np.array([3,4,5,6])#,7,8,9,10,20,30,40])
    
    pixfile = file
    hdu = fits.open(pixfile)
    dat = hdu[1].data
    data = fits.ImageHDU(hdu[1].data.field('FLUX')[:]).data#np.copy(testdata)#
    time = dat["TIME"] + 2454833.0
    Qual = hdu[1].data.field('QUALITY')

    funny_keywords = {'1CTYP4': 'CTYPE1',
                      '2CTYP4': 'CTYPE2',
                      '1CRPX4': 'CRPIX1',
                      '2CRPX4': 'CRPIX2',
                      '1CRVL4': 'CRVAL1',
                      '2CRVL4': 'CRVAL2',
                      '1CUNI4': 'CUNIT1',
                      '2CUNI4': 'CUNIT2',
                      '1CDLT4': 'CDELT1',
                      '2CDLT4': 'CDELT2',
                      '11PC4': 'PC1_1',
                      '12PC4': 'PC1_2',
                      '21PC4': 'PC2_1',
                      '22PC4': 'PC2_2'}
    mywcs = {}
    for oldkey, newkey in funny_keywords.items():
        mywcs[newkey] = hdu[1].header[oldkey]
    wcs = WCS(mywcs)
    
    for D in range(len(Duration)):
        Detections = np.zeros((len(Mag),data.shape[1],data.shape[2]))
        Seeded = np.zeros((len(Mag),data.shape[1],data.shape[2]))
        Limit = np.zeros((data.shape[1],data.shape[2]))
        # arrays to initialise multiple concurrent tests
        x = [np.arange(0,data.shape[1],2),np.arange(1,data.shape[1],2)]
        y = [np.arange(0,data.shape[2],2),np.arange(1,data.shape[2],2)]
        #t = randint(100,data.shape[0]-100,size=(data.shape[2]))
        
        for k in range(len(Mag)):
        
            Seed = K2Steptest(Mag[k],Duration[D],time,Start)

            for i in range(2):
                for j in range(2):
                    X, Y = np.meshgrid(x[i],y[j],indexing='ij')
                    testdata = np.copy(data)
                    testdata[Start:,X,Y] += Seed[Start:,None,None]
                    detec = K2tranPix(testdata,time,Qual,wcs,X,Y,Start)
                    testdata=[]
                    Seeded[k,X,Y] += 1
                    if (detec == 1).any:
                        Detections[k,np.where(detec == 1)[0],np.where(detec == 1)[1]] += 1
            plt.figure()
            plt.imshow(Detections[k],origin = 'lower')       
        for i in range(data.shape[1]):
            for j in range(data.shape[2]):
                Index = np.where(Detections[:,i,j] == 1)[0]
                if len(Index) > 0:
                    Limit[i,j] = Mag[Index[-1]]
        Limit[Limit == 0] = np.nan
        plt.figure()
        plt.imshow(Limit,origin = 'lower')  
        plt.colorbar()
        save = path+file.split('/')[-1].split('-')[0]+str(Start)+'_'+str(Duration[D])

        try:

            if not os.path.exists(path+file.split('/')[-1]):
                os.makedirs(path+file.split('/')[-1].split('ktwo')[0])
            np.save(save,Limit)
        except FileExistsError:
            np.save(save,Limit)
        
    
    return